In [18]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import os
import imageio.v2 as imageio 

# Parâmetros
nx = 100
ny = nx
dx, dy = 1 / nx, 1 / ny
u_max = 0.1 / nx
dt = dx / u_max
re = 5
alpha = dx / dt * nx / re
omega = 1 / (3 * alpha + 0.5)
rho0 = 1
t = int(1300)
G = -5
rho_cr = np.log(2)
rho_liq = 1.95
rho_gas = 0.15
count=0

output_dir = os.path.join(os.path.expanduser("~"), "Documents", "PraticaLBM","Densidades")
os.makedirs(output_dir, exist_ok=True)

# Componentes de velocidade e pesos
cx = np.array([0, 1, 0, -1, 0, 1, -1, -1, 1])
cy = np.array([0, 0, 1, 0, -1, 1, 1, -1, -1])
w = np.array([0.4444444444, 0.1111111111, 0.1111111111, 0.1111111111, 
              0.1111111111, 0.0277777777, 0.0277777777, 0.0277777777, 0.0277777777])

# Definição da malha
X, Y = np.meshgrid(np.linspace(dx / 2, dx / 2 + (nx - 1) * dx, nx, dtype='float64'),
                   np.linspace(dy / 2, dy / 2 + (ny - 1) * dy, ny, dtype='float64'), indexing='ij')

# Inicialização das variáveis
f = np.zeros((9, nx, ny), dtype='float64')
fp = np.zeros((9, nx, ny), dtype='float64')
rho = rho_cr + 0.1 * np.random.rand(nx, ny)
print(f"Min rho: {rho.min()}, Max rho: {rho.max()}")
u = np.zeros((nx, ny), dtype='float64')
v = np.zeros((nx, ny), dtype='float64')
uf = np.zeros((nx, ny), dtype='float64')
vf = np.zeros((nx, ny), dtype='float64')
forcx = np.zeros((nx, ny))
forcy = np.zeros((nx, ny))

# Função de força
def force():
    global forcx, forcy
    forcx = np.zeros((nx, ny))
    forcy = np.zeros((nx, ny))
    for j in range(ny):
        for i in range(nx):
            forcxs = 0.0
            forcys = 0.0
            for k in range(8):
                k=k+1
                newx = (i + cx[k] + nx) % nx
                newy = (j + cy[k] + ny) % ny
                psi = 1 - np.exp(-rho[newx, newy])
                forcxs -= G * w[k] * psi * cx[k]
                forcys -= G * w[k] * psi * cy[k]
            forcx[i, j] = (1 - np.exp(-rho[i, j])) * forcxs
            forcy[i, j] = (1 - np.exp(-rho[i, j])) * forcys

# Funções de colisão e transmissão
def fcolisao():
    global fp
    # Precompute terms
    t1 = uf**2 + vf**2  # Squared velocities
    t2 = np.einsum('i,jk->ijk', cx, uf) + np.einsum('i,jk->ijk', cy, vf)  # Dot product of velocity and discrete velocities
    
    # Compute equilibrium distribution function (feq)
    feq = np.zeros_like(f)
    for i in range(9):
        feq[i] = w[i] * rho * (1 + 3 * t2[i] + 4.5 * t2[i]**2 - 1.5 * t1)

    # Compute forcing terms
    ffx = (3 * (cx[:, None, None] - uf) + 9 * cx[:, None, None] * t2) * forcx
    ffy = (3 * (cy[:, None, None] - vf) + 9 * cy[:, None, None] * t2) * forcy
    fftot = w[:, None, None] * (1 - 0.5 * omega) * (ffx + ffy)
    
    # Update population
    fp = f * (1 - omega) + omega * feq + fftot


def ftransmissao():
    global f
    for i in range(9):
        f[i] = np.roll(np.roll(fp[i], int(cx[i]), axis=0), int(cy[i]), axis=1)

def boundary():
    global f
    
    f[3, :, ny - 1] = f[3, :, 0]
    f[7, :, ny - 1] = f[7, :, 0]
    f[6, :, ny - 1] = f[6, :, 0]
    
    f[0, 0, :] = f[0, nx - 1, :]
    f[4, 0, :] = f[4, nx - 1, :]
    f[7, 0, :] = f[7, nx - 1, :]

    f[2, nx - 1, :] = f[2, 0, :]
    f[6, nx - 1, :] = f[6, 0, :]
    f[5, nx - 1, :] = f[5, 0, :]

    f[1, :, 0] = f[1, :, ny - 1]
    f[4, :, 0] = f[4, :, ny - 1]
    f[5, :, 0] = f[5, :, ny - 1]

t1 = uf**2 + vf**2  

for count in range(t):
    force()
    fcolisao()
    ftransmissao()
    boundary()
    
    # Calcula densidade e velocidades
    rho = np.einsum('ijk->jk', f)
    u = (f[1] + f[5] + f[8] - (f[3] + f[6] + f[7])) / rho
    v = (f[2] + f[5] + f[6] - (f[4] + f[7] + f[8])) / rho
    uf = u + 0.5 * forcx / rho
    vf = v + 0.5 * forcy / rho
    
    # Salva imagem a cada 10 passos de tempo
    if count % 10 == 0:  # Salva no 0, 10, 20, ...
        plt.figure(figsize=(6, 6))
        plt.imshow(rho, cmap="viridis", origin="lower")
        plt.colorbar(label="Densidade")
        plt.title(f"Frame {count}")
        plt.savefig(f"{output_dir}frame_{int(count / 10):03d}.png")
        plt.close()

# Criar animação com imagens salvas
import glob

with imageio.get_writer("densidade_animacao.gif", mode="I", duration=0.1) as writer:
    frame_files = sorted(glob.glob(f"{output_dir}frame_*.png"))  # Ordena os frames corretamente
    for filename in frame_files:
        image = imageio.imread(filename)
        writer.append_data(image)

print("Animação salva como 'densidade_animacao.gif'")

ModuleNotFoundError: No module named 'matplotlib'

In [4]:
### CODIGO ORIGINAL EM FUNCIONAMENTO  - NAO MEXER !!!!!!!!!!!

import numpy as np
import os
import time
import matplotlib.pyplot as plt


# Parâmetros do lattice
nx = 64  # número de nós ao longo do eixo x
ny = 64  # número de nós ao longo do eixo y
nsteps = 20000  # número de passos de tempo
noutput = 100  # intervalo de saída de dados (dados escritos em "/data")
nfluids = 1  # número de componentes do fluido (escolha 1 ou 2)
tauA = 1  # tempo de relaxação para o fluido A
tauB = 1  # tempo de relaxação para o fluido B (usado apenas se nfluids = 2)

# Parâmetros de Shan-Chen
gA = -4.7  # força de auto-interação do fluido A
gB = 0.  # força de auto-interação do fluido B (usado apenas se nfluids = 2)
gAB = 6.  # força de interação dos fluidos A e B (usado apenas se nfluids = 2)
rho0 = 1.  # densidade de referência para o potencial pseudo (geralmente definido como 1)
rhol = 2.1  # densidade inicial do líquido
rhog = 0.15  # densidade inicial do gás
radius = 15.  # raio inicial da gota

# Parâmetros fixos; NÃO ALTERAR
npop = 9  # número de populações
cx =[0, 1, 0, -1, 0, 1, -1, -1, 1] # componentes x dos vetores do lattice
cy =[0, 0, 1,  0, -1, 1,  1, -1, -1]  # componentes y dos vetores do lattice
weight = [4./9., 1./9., 1./9., 1./9., 1./9., 1./36., 1./36., 1./36., 1./36.]


# Matrizes
g = np.zeros((nfluids, nfluids))  # forças de interação
rho = np.zeros((nfluids, nx, ny))  # densidade
press = np.zeros((nx, ny))  # pressão
ux = np.zeros((nx, ny))  # componente x da velocidade do fluido
uy = np.zeros((nx, ny))  # componente y da velocidade do fluido
psi_s = np.zeros((nx, ny))  # paralelizando calculos
psi_ss = np.zeros((nx, ny))  # paralelizando calculos
psinb = np.zeros((nx, ny))  # paralelizando calculos
# fxtemp=np.zeros((nx, ny))  # paralelizando calculos
# fytemp=np.zeros((nx, ny))  # paralelizando calculos
Fx = np.zeros((nfluids, nx, ny))  # componente x da força de Shan-Chen
Fy = np.zeros((nfluids, nx, ny))  # componente y da força de Shan-Chen
feq = np.zeros((nfluids, npop))  # populações de equilíbrio
forcing = np.zeros((nfluids, npop))  # populações de força
tau = np.zeros(nfluids)  # tempos de relaxação
f1 = np.zeros((nfluids, nx, ny, npop))  # populações (antigo)
f2 = np.zeros((nfluids, nx, ny, npop))  # populações (novo)

output_dir = os.path.join(os.path.expanduser("~"), "Documents", "PraticaLBM","Densidades")
os.makedirs(output_dir, exist_ok=True)



# Função para computar a densidade em todos os pontos

def compute_density(pop):
    global rho
    rho = np.zeros((nfluids, nx, ny))  # A matriz de densidade já é inicializada fora da função
    for s in range(nfluids):
        rho[s] = np.sum(pop, axis=3).reshape(nx, ny)  # Soma todas as direções em cada nó


def computePressure():
    # Inicializa a matriz de pressão com zeros
    press.fill(0.0)
    
    # Calcula o termo de correção de forma vetorizada
    for s in range(nfluids):
        press += rho[s] / 3.0
        for ss in range(nfluids):
            press += (g[s, ss] * 
                      psi(rho[s]) * 
                      psi(rho[ss]) / 6.0)

def psi(densities):
    return rho0 * (1 - np.exp(-densities / rho0))
                    

def computeSCForces():
    # Inicializa as forças SC
    Fx.fill(0.0)
    Fy.fill(0.0)
    xvet=np.arange(nx)
    x2=np.zeros((nx,npop))
    yvet=np.arange(ny)
    y2=np.zeros((ny,npop))
    for x in range(nx):
        for i in range(npop):
            x2[x,i]=xvet[(x + cx[i]) % nx]
            y2[x,i]=yvet[(x + cy[i]) % ny]  ##como y=x nesse caso, usamos o mesmo loop
    # Computa as forças de Shan-Chen (SC)
    for x in range(nx):
        for y in range(ny):
            for s in range(nfluids):
                for ss in range(nfluids):
                    fxtemp=0
                    fytemp=0
                    for i in range(1, npop):  # Ignorar i=0 (sem força)
                        ##fx e fy temp tavam aqui
                        x2var=int(x2[x,i])
                        y2var=int(y2[y,i])
                        psinb[x,y] = rho0 * (1 - np.exp(-rho[s,x2var,y2var] / rho0))  # Psi do vizinho
                        fxtemp += weight[i] * cx[i] * psinb[x,y]
                        fytemp += weight[i] * cy[i] * psinb[x,y]
                        
                    psiloc = rho0 * (1 - np.exp(-rho[s,x,y] / rho0)) # Psi local
                    Fx[s, x, y] -= g[s, ss] * psiloc * fxtemp
                    Fy[s, x, y] -= g[s, ss] * psiloc * fytemp

    
def computeTotalDensity(x, y):
    # Soma a densidade de todos os fluidos no ponto (x, y)
    return np.sum(rho[:, x, y])




# def computePressure():
#     # Inicializa a pressão
#     press.fill(0.0)
#     psi_s.fill(0.0)
#     psi_ss.fill(0.0)
#     press.fill(0.0)
#     press[:, :] += np.sum(rho[:, :, :]) / 3.0
#     # Soma contribuições para a pressão
#     for s in range(nfluids):
#         for ss in range(nfluids):
#             psi_s[:,:] = rho0 * (1 - np.exp(-rho[s, :, :] / rho0))
#             psi_ss[:,:] = rho0 * (1 - np.exp(-rho[ss, :, :] / rho0))
#             press[:, :] += (g[s, ss] * psi_s[:,:] * psi_ss[:,:]) / 6.0

def computePressure():
    # Inicializa a pressão
    press.fill(0.0)
    psi_s = rho0 * (1 - np.exp(-rho / rho0))  # Calcula psi para todos os fluidos de uma vez
    psi_ss = psi_s  # psi_ss é o mesmo que psi_s, já que a operação é simétrica
    press[:, :] += np.sum(rho, axis=0) / 3.0  # Soma as densidades divididas por 3.0

    # Soma contribuições para a pressão com operações vetorizadas
    press[:, :] += np.sum(g[:, :, None, None] * psi_s[:, None, :, :] * psi_ss[None, :, :, :], axis=(0, 1)) / 6.0
                    

# def computeVelocity(pop):
#     for x in range(nx):
#         for y in range(ny):
#             ux[x, y] = 0.0
#             uy[x, y] = 0.0
#             for s in range(nfluids):
#                 # Calcula contribuições da população
#                 ux[x, y] += (pop[s, x, y, 1] - pop[s, x, y, 3] +
#                              pop[s, x, y, 5] - pop[s, x, y, 6] -
#                              pop[s, x, y, 7] + pop[s, x, y, 8])
                
#                 uy[x, y] += (pop[s, x, y, 2] - pop[s, x, y, 4] +
#                              pop[s, x, y, 5] + pop[s, x, y, 6] -
#                              pop[s, x, y, 7] - pop[s, x, y, 8])
                
#                 # Adiciona força
#                 ux[x, y] += 0.5 * Fx[s, x, y]
#                 uy[x, y] += 0.5 * Fy[s, x, y]
            
#             # Normaliza pela densidade total
#             dens = computeTotalDensity(x, y)
#             ux[x, y] /= dens
#             uy[x, y] /= dens

def computeVelocity(pop):
    ux.fill(0.0)
    uy.fill(0.0)
    for s in range(nfluids):
        # Calcula contribuições da população
        ux[:,:] += (pop[s, :, :, 1] - pop[s, :, :, 3] +
                     pop[s, :, :, 5] - pop[s, :, :, 6] -
                     pop[s, :, :, 7] + pop[s, :, :, 8])

        uy[:, :] += (pop[s, :, :, 2] - pop[s, :, :, 4] +
                     pop[s, :, :, 5] + pop[s, :, :, 6] -
                     pop[s, :, :, 7] - pop[s, :, :, 8])

        # Adiciona força
        ux[:, :] += 0.5 * Fx[s, :, :]
        uy[:, :] += 0.5 * Fy[s, :, :]

# Normaliza pela densidade total
    for x in range(nx):
        for y in range(ny):
            dens = computeTotalDensity(x,y)
            ux[x, y] /= dens
            uy[x, y] /= dens

            

def equilibrium(s,x,y):
    dens = rho[s,x,y]
    vx = ux[x,y]
    vy = uy[x,y]
    usq = vx * vx + vy * vy
  
    feq[s][0] = weight[0] * dens * (1. - 1.5 * usq)
    feq[s][1] = weight[1] * dens * (1. + 3. * vx + 4.5 * vx * vx - 1.5 * usq)
    feq[s][2] = weight[2] * dens * (1. + 3. * vy + 4.5 * vy * vy - 1.5 * usq)
    feq[s][3] = weight[3] * dens * (1. - 3. * vx + 4.5 * vx * vx - 1.5 * usq)
    feq[s][4] = weight[4] * dens * (1. - 3. * vy + 4.5 * vy * vy - 1.5 * usq)
    feq[s][5] = weight[5] * dens * (1. + 3. * ( vx + vy) + 4.5 * ( vx + vy) * ( vx + vy) - 1.5 * usq)
    feq[s][6] = weight[6] * dens * (1. + 3. * (-vx + vy) + 4.5 * (-vx + vy) * (-vx + vy) - 1.5 * usq)
    feq[s][7] = weight[7] * dens * (1. + 3. * (-vx - vy) + 4.5 * ( vx + vy) * ( vx + vy) - 1.5 * usq)
    feq[s][8] = weight[8] * dens * (1. + 3. * ( vx - vy) + 4.5 * ( vx - vy) * ( vx - vy) - 1.5 * usq)
  

  
def initialisation(f1, f2):
    # Definir viscosidade
    tau[0] = tauA
    if nfluids == 2:
        tau[1] = tauB

    # Definir força de interação
    g[0, 0] = gA
    if nfluids == 2:
        g[1, 1] = gB
        g[0, 1] = gAB
        g[1, 0] = gAB

    # Inicialização para 1 fluido
    if nfluids == 1:
        x_grid, y_grid = np.meshgrid(np.arange(nx), np.arange(ny), indexing="ij")
        distances = (y_grid - ny / 2.0) ** 2 + (x_grid - nx / 2.0) ** 2
        rho[0] = np.where(distances <= radius ** 2, rhol, rhog)

    # Inicialização para 2 fluidos
    elif nfluids == 2:
        mid_y = ny // 2
        rho[0, :, :mid_y] = 0.1
        rho[1, :, :mid_y] = 0.9
        rho[0, :, mid_y:] = 0.9
        rho[1, :, mid_y:] = 0.1

    # Configurar o campo de velocidade inicial (velocidade zero em todos os lugares)
    ux.fill(0.0)
    uy.fill(0.0)

    # Calcular as distribuições de equilíbrio e inicializar as populações
    for s in range(nfluids):
        for y in range(ny):
            for x in range(nx):
                equilibrium(s, x, y)
                f1[s, x, y, :] = feq[s, :]
                f2[s, x, y, :] = feq[s, :]    
    
def push(f1, f2):
    for s in range(nfluids):
        omega = 1.0 / tau[s]

        for x in range(nx):  
            for y in range(ny):  
                for i in range(npop):
                    forcing[s,i] = weight[i] * (1.0 - 0.5 * omega) * \
                        ((3.0 * (cx[i] - ux[x, y]) + 9.0 * cx[i] * (cx[i] * ux[x, y] + cy[i] * uy[x, y])) * Fx[s, x, y] +
                         (3.0 * (cy[i] - uy[x, y]) + 9.0 * cy[i] * (cx[i] * ux[x, y] + cy[i] * uy[x, y])) * Fy[s, x, y])

                # Compute equilibrium distributions
                equilibrium(s, x, y)

                # Collide and propagate
                for i in range(npop):
                    x2 = (x + cx[i] + nx) % nx  # Periodic boundary in x
                    y2 = (y + cy[i] + ny) % ny  # Periodic boundary in y
                    f2[s, x2, y2, i] = f1[s, x, y, i] * (1.0 - omega) + feq[s, i] * omega + forcing[s,i]

    return

                 
def write_file(name, data, n):
    #Função auxiliar para salvar dados em disco.
    name = os.path.expanduser(os.path.join("~", "Documents", "PraticaLBM","Densidades"))
    # Verifique se o diretório existe, caso contrário, crie-o
    os.makedirs(os.path.dirname(name), exist_ok=True)
    with open(name, "w") as fout:
        for y in range(ny):
            for x in range(nx):
                fout.write(f"{data[y * nx + x]} ")
            fout.write("\n")
        fout.write("\n")


def write_profiles(step):
    """
    Função para salvar os perfis (densidade, pressão, velocidades) no disco.
    """
    # Converter o número do passo em string com zero-padding
    step_str = str(step).zfill(5)

    # Salvar os perfis de pressão e velocidades
    press_filename = f"press_{step_str}.dat"
    velx_filename = f"velx_{step_str}.dat"
    vely_filename = f"vely_{step_str}.dat"

    write_file(press_filename, press, nx * ny)
    write_file(velx_filename, ux, nx * ny)
    write_file(vely_filename, uy, nx * ny)

    # Salvar os perfis de densidade para cada componente de fluido
    for s in range(nfluids):
        density_filename = f"density_comp{s}_{step_str}.dat"
        write_file(density_filename, rho[s], nx * ny)


def calculate_delta_pressure():
    """
    Calcula a pressão de Laplace e imprime densidades, diferenças de pressão, 
    raio da gota e tensão superficial.
    """
    rho_gas = 0.0
    rho_liq = 0.0
    press_gas = 0.0
    press_liq = 0.0

    # Pressão média do gás em um quadrado de 4 nós de malha
    for x in range(4):
        for y in range(4):
            rho_gas += rho[0,x,y]
            press_gas += press[x,y]

    # Pressão média do líquido em um quadrado de 4 nós de malha no centro
    for x in range(nx // 2 - 2, nx // 2 + 2):
        for y in range(ny // 2 - 2, ny // 2 + 2):
            rho_liq += rho[0,x,y]
            press_liq += press[x,y]

    # Calcula médias
    rho_gas /= 16.0
    rho_liq /= 16.0
    press_gas /= 16.0
    press_liq /= 16.0

    # Diferença de pressão e densidade média
    delta_press = press_liq - press_gas
    rho_av = 0.5 * (rho_gas + rho_liq)
    y = ny // 2
    rad = None

    # Determina o raio da gota
    for x in range(nx // 2):
        if rho[0,x,y] > rho_av:
            drho = rho[0,x,y] - rho[0,x-1,y]
            dx = (rho_av - rho[0,x-1,y]) / drho
            rad = (nx / 2.0 - x) + (1.0 - dx)
            break

    # Calcula a tensão superficial
    gamma = delta_press * rad

    # Imprime os resultados
    print(f"  Densities: rho_g = {rho_gas}, rho_l = {rho_liq}")
    print(f"  Pressure difference: dp = {delta_press}")
    print(f"  Droplet radius: r = {rad}")
    print(f"  Surface tension: gamma = {gamma}")

    return


# Função para imprimir status, fora do Numba
def print_status(step, elapsed_time):
    print("Running time step {}".format(step))
    print("Overall CPU time required: {:.2f} sec".format(elapsed_time))
    mlups = nx * ny * nsteps / (elapsed_time * 1e6)
    print("This corresponds to {:.2f} MLUPS".format(mlups))

# Função para criar animação, fora do Numba
def create_animation(output_dir):
    with imageio.get_writer("densidade_animacao.gif", mode="I", duration=0.1) as writer:
        frame_files = sorted(glob.glob("{}frame_*.png".format(output_dir)))  # Ordena os frames corretamente
        for filename in frame_files:
            image = imageio.imread(filename)
            writer.append_data(image)

    print("Animação salva como 'densidade_animacao.gif'")

def main():
    # Configurar cronômetro da simulação
    start = time.time()

    # Alocar memória para populações
    f1 = np.zeros((nfluids, nx, ny, npop))  # Populações antigas
    f2 = np.zeros((nfluids, nx, ny, npop))  # Populações novas
    # Inicializar domínio
    initialisation(f1, f2)

    # Loop principal
    for step in range(nsteps + 1):
        # Calcular campo de densidade
        compute_density(f1)

        # Calcular forças SC
        computeSCForces()

        # Calcular velocidade baricêntrica
        computeVelocity(f1)

        # Colidir e propagar
        push(f1, f2)

        # Trocar populações antigas e novas
        f1, f2 = f2, f1

        # Escrever dados em disco/console
        if step % noutput == 0:
            print_status(step, time.time() - start)
            computePressure()
            #write_profiles(step)
            calculate_delta_pressure()
            rho_summed = np.einsum('sxy->xy', rho)
            rho_matriz = rho_summed.reshape(nx, ny)
            plt.figure(figsize=(6, 6))
            plt.imshow(rho_matriz, cmap="viridis", origin="lower")
            plt.colorbar(label="Densidade")
            plt.title("Frame {}".format(step))
            plt.savefig("{}frame_{:03d}.png".format(output_dir, int(step / 10)))
            plt.close()

    # Exibir estatísticas de desempenho
    finish = time.time()
    elapsed_time = finish - start
    print_status(nsteps, elapsed_time)  # Imprimir estatísticas no final

# Chamar a função principal
if __name__ == "__main__":
    main()

#Criar animação após a execução do cálculo
create_animation(output_dir)

### CODIGO ORIGINAL EM FUNCIONAMENTO  - NAO MEXER !!!!!!!!!!!

Running time step 0
Overall CPU time required: 0.26 sec
This corresponds to 313.30 MLUPS
  Densities: rho_g = 0.14999999999999997, rho_l = 2.1000000000000005
  Pressure difference: dp = 0.06196697888838586
  Droplet radius: r = 15.5
  Surface tension: gamma = 0.9604881727699808
Running time step 100
Overall CPU time required: 22.11 sec
This corresponds to 3.71 MLUPS
  Densities: rho_g = 0.24713655112712946, rho_l = 0.2478824713550728
  Pressure difference: dp = 3.626802089425063e-05
  Droplet radius: r = 20.79283522111776
  Surface tension: gamma = 0.0007541149822502093
Running time step 200
Overall CPU time required: 43.91 sec
This corresponds to 1.87 MLUPS
  Densities: rho_g = 0.16512749875777133, rho_l = 0.3445171771682526
  Pressure difference: dp = 0.008617418393895147
  Droplet radius: r = 19.274791687648612
  Surface tension: gamma = 0.16609894442764045
Running time step 300
Overall CPU time required: 66.04 sec
This corresponds to 1.24 MLUPS
  Densities: rho_g = 0.17966939977902

KeyboardInterrupt: 

In [13]:
### CODIGO MODIFICADO PARA INICIALIZAÇÃO ALEATÓRIA

import numpy as np
import os
import time
import matplotlib.pyplot as plt


# Parâmetros do lattice
nx = 64  # número de nós ao longo do eixo x
ny = 64  # número de nós ao longo do eixo y
nsteps = 2000  # número de passos de tempo
noutput = 5  # intervalo de saída de dados (dados escritos em "/data")
nfluids = 1  # número de componentes do fluido (escolha 1 ou 2)
tauA = 1  # tempo de relaxação para o fluido A
tauB = 1  # tempo de relaxação para o fluido B (usado apenas se nfluids = 2)

# Parâmetros de Shan-Chen
gA = -4.7  # força de auto-interação do fluido A
gB = 0.  # força de auto-interação do fluido B (usado apenas se nfluids = 2)
gAB = 6.  # força de interação dos fluidos A e B (usado apenas se nfluids = 2)
rho0 = 1.  # densidade de referência para o potencial pseudo (geralmente definido como 1)
rhol = 2.1  # densidade inicial do líquido
rhog = 0.15  # densidade inicial do gás
radius = 15.  # raio inicial da gota

# Parâmetros fixos; NÃO ALTERAR
npop = 9  # número de populações
cx =[0, 1, 0, -1, 0, 1, -1, -1, 1] # componentes x dos vetores do lattice
cy =[0, 0, 1,  0, -1, 1,  1, -1, -1]  # componentes y dos vetores do lattice
weight = [4./9., 1./9., 1./9., 1./9., 1./9., 1./36., 1./36., 1./36., 1./36.]


# Matrizes
g = np.zeros((nfluids, nfluids))  # forças de interação
rho = np.zeros((nfluids, nx, ny))  # densidade
press = np.zeros((nx, ny))  # pressão
ux = np.zeros((nx, ny))  # componente x da velocidade do fluido
uy = np.zeros((nx, ny))  # componente y da velocidade do fluido
psi_s = np.zeros((nx, ny))  # paralelizando calculos
psi_ss = np.zeros((nx, ny))  # paralelizando calculos
psinb = np.zeros((nx, ny))  # paralelizando calculos
# fxtemp=np.zeros((nx, ny))  # paralelizando calculos
# fytemp=np.zeros((nx, ny))  # paralelizando calculos
Fx = np.zeros((nfluids, nx, ny))  # componente x da força de Shan-Chen
Fy = np.zeros((nfluids, nx, ny))  # componente y da força de Shan-Chen
feq = np.zeros((nfluids, npop))  # populações de equilíbrio
forcing = np.zeros((nfluids, npop))  # populações de força
tau = np.zeros(nfluids)  # tempos de relaxação
f1 = np.zeros((nfluids, nx, ny, npop))  # populações (antigo)
f2 = np.zeros((nfluids, nx, ny, npop))  # populações (novo)
fraction_fluid=0.5  ## fração de fluido

output_dir = os.path.join(os.path.expanduser("~"), "Documents", "PraticaLBM","Densidades")
os.makedirs(output_dir, exist_ok=True)


# Função para computar a densidade em todos os pontos

def compute_density(pop):
    global rho
    rho = np.zeros((nfluids, nx, ny))  # A matriz de densidade já é inicializada fora da função
    for s in range(nfluids):
        rho[s] = np.sum(pop, axis=3).reshape(nx, ny)  # Soma todas as direções em cada nó


def computePressure():
    # Inicializa a matriz de pressão com zeros
    press.fill(0.0)
    
    # Calcula o termo de correção de forma vetorizada
    for s in range(nfluids):
        press += rho[s] / 3.0
        for ss in range(nfluids):
            press += (g[s, ss] * 
                      psi(rho[s]) * 
                      psi(rho[ss]) / 6.0)

def psi(densities):
    return rho0 * (1 - np.exp(-densities / rho0))
                    

def computeSCForces():
    # Inicializa as forças SC
    Fx.fill(0.0)
    Fy.fill(0.0)
    xvet=np.arange(nx)
    x2=np.zeros((nx,npop))
    yvet=np.arange(ny)
    y2=np.zeros((ny,npop))
    for x in range(nx):
        for i in range(npop):
            x2[x,i]=xvet[(x + cx[i]) % nx]
            y2[x,i]=yvet[(x + cy[i]) % ny]  ##como y=x nesse caso, usamos o mesmo loop
    # Computa as forças de Shan-Chen (SC)
    for x in range(nx):
        for y in range(ny):
            for s in range(nfluids):
                for ss in range(nfluids):
                    fxtemp=0
                    fytemp=0
                    for i in range(1, npop):  # Ignorar i=0 (sem força)
                        ##fx e fy temp tavam aqui
                        x2var=int(x2[x,i])
                        y2var=int(y2[y,i])
                        psinb[x,y] = rho0 * (1 - np.exp(-rho[s,x2var,y2var] / rho0))  # Psi do vizinho
                        fxtemp += weight[i] * cx[i] * psinb[x,y]
                        fytemp += weight[i] * cy[i] * psinb[x,y]
                        
                    psiloc = rho0 * (1 - np.exp(-rho[s,x,y] / rho0)) # Psi local
                    Fx[s, x, y] -= g[s, ss] * psiloc * fxtemp
                    Fy[s, x, y] -= g[s, ss] * psiloc * fytemp

    
def computeTotalDensity(x, y):
    # Soma a densidade de todos os fluidos no ponto (x, y)
    return np.sum(rho[:, x, y])


def computePressure():
    # Inicializa a pressão
    press.fill(0.0)
    psi_s = rho0 * (1 - np.exp(-rho / rho0))  # Calcula psi para todos os fluidos de uma vez
    psi_ss = psi_s  # psi_ss é o mesmo que psi_s, já que a operação é simétrica
    press[:, :] += np.sum(rho, axis=0) / 3.0  # Soma as densidades divididas por 3.0

    # Soma contribuições para a pressão com operações vetorizadas
    press[:, :] += np.sum(g[:, :, None, None] * psi_s[:, None, :, :] * psi_ss[None, :, :, :], axis=(0, 1)) / 6.0
                    
def computeVelocity(pop):
    ux.fill(0.0)
    uy.fill(0.0)
    for s in range(nfluids):
        # Calcula contribuições da população
        ux[:,:] += (pop[s, :, :, 1] - pop[s, :, :, 3] +
                     pop[s, :, :, 5] - pop[s, :, :, 6] -
                     pop[s, :, :, 7] + pop[s, :, :, 8])

        uy[:, :] += (pop[s, :, :, 2] - pop[s, :, :, 4] +
                     pop[s, :, :, 5] + pop[s, :, :, 6] -
                     pop[s, :, :, 7] - pop[s, :, :, 8])

        # Adiciona força
        ux[:, :] += 0.5 * Fx[s, :, :]
        uy[:, :] += 0.5 * Fy[s, :, :]

# Normaliza pela densidade total
    for x in range(nx):
        for y in range(ny):
            dens = computeTotalDensity(x,y)
            ux[x, y] /= dens
            uy[x, y] /= dens

            

def equilibrium(s,x,y):
    dens = rho[s,x,y]
    vx = ux[x,y]
    vy = uy[x,y]
    usq = vx * vx + vy * vy
  
    feq[s][0] = weight[0] * dens * (1. - 1.5 * usq)
    feq[s][1] = weight[1] * dens * (1. + 3. * vx + 4.5 * vx * vx - 1.5 * usq)
    feq[s][2] = weight[2] * dens * (1. + 3. * vy + 4.5 * vy * vy - 1.5 * usq)
    feq[s][3] = weight[3] * dens * (1. - 3. * vx + 4.5 * vx * vx - 1.5 * usq)
    feq[s][4] = weight[4] * dens * (1. - 3. * vy + 4.5 * vy * vy - 1.5 * usq)
    feq[s][5] = weight[5] * dens * (1. + 3. * ( vx + vy) + 4.5 * ( vx + vy) * ( vx + vy) - 1.5 * usq)
    feq[s][6] = weight[6] * dens * (1. + 3. * (-vx + vy) + 4.5 * (-vx + vy) * (-vx + vy) - 1.5 * usq)
    feq[s][7] = weight[7] * dens * (1. + 3. * (-vx - vy) + 4.5 * ( vx + vy) * ( vx + vy) - 1.5 * usq)
    feq[s][8] = weight[8] * dens * (1. + 3. * ( vx - vy) + 4.5 * ( vx - vy) * ( vx - vy) - 1.5 * usq)
  

  
def initialisation(f1, f2):
    # Definir viscosidade
    tau[0] = tauA
    if nfluids == 2:
        tau[1] = tauB

    # Definir força de interação
    g[0, 0] = gA
    if nfluids == 2:
        g[1, 1] = gB
        g[0, 1] = gAB
        g[1, 0] = gAB

    # Inicialização para 1 fluido
    if nfluids == 1:
        # Gerar uma máscara aleatória para densidades de fluido e gás
        random_mask = np.random.rand(nx, ny) < fraction_fluid
        rho[0] = np.where(random_mask, rhol, rhog)

    # Inicialização para 2 fluidos
    elif nfluids == 2:
        mid_y = ny // 2
        rho[0, :, :mid_y] = 0.1
        rho[1, :, :mid_y] = 0.9
        rho[0, :, mid_y:] = 0.9
        rho[1, :, mid_y:] = 0.1

    # Configurar o campo de velocidade inicial (velocidade zero em todos os lugares)
    ux.fill(0.0)
    uy.fill(0.0)

    # Calcular as distribuições de equilíbrio e inicializar as populações
    for s in range(nfluids):
        for y in range(ny):
            for x in range(nx):
                equilibrium(s, x, y)
                f1[s, x, y, :] = feq[s, :]
                f2[s, x, y, :] = feq[s, :]
                
def push(f1, f2):
    for s in range(nfluids):
        omega = 1.0 / tau[s]

        for x in range(nx):  
            for y in range(ny):  
                for i in range(npop):
                    forcing[s,i] = weight[i] * (1.0 - 0.5 * omega) * \
                        ((3.0 * (cx[i] - ux[x, y]) + 9.0 * cx[i] * (cx[i] * ux[x, y] + cy[i] * uy[x, y])) * Fx[s, x, y] +
                         (3.0 * (cy[i] - uy[x, y]) + 9.0 * cy[i] * (cx[i] * ux[x, y] + cy[i] * uy[x, y])) * Fy[s, x, y])

                # Compute equilibrium distributions
                equilibrium(s, x, y)

                # Collide and propagate
                for i in range(npop):
                    x2 = (x + cx[i] + nx) % nx  # Periodic boundary in x
                    y2 = (y + cy[i] + ny) % ny  # Periodic boundary in y
                    f2[s, x2, y2, i] = f1[s, x, y, i] * (1.0 - omega) + feq[s, i] * omega + forcing[s,i]

    return

                 
# def write_file(name, data, n):
#     #Função auxiliar para salvar dados em disco.
#     name = os.path.expanduser(os.path.join("~", "Documents", "PraticaLBM","Densidades", name))
#     # Verifique se o diretório existe, caso contrário, crie-o
#     os.makedirs(os.path.dirname(name), exist_ok=True)
#     with open(name, "w") as fout:
#         for y in range(ny):
#             for x in range(nx):
#                 fout.write(f"{data[y * nx + x]} ")
#             fout.write("\n")
#         fout.write("\n")

def write_file(name, data, n):
    # Use output_dir para consistência
    file_path = os.path.join(output_dir, name)
    print(file_path)
    os.makedirs(output_dir, exist_ok=True)  # Garantir que o diretório exista
    with open(file_path, "w") as fout:
        for y in range(ny):
            for x in range(nx):
                fout.write(f"{data[y * nx + x]} ")
            fout.write("\n")
        fout.write("\n")


def write_profiles(step):
    """
    Função para salvar os perfis (densidade, pressão, velocidades) no disco.
    """
    # Converter o número do passo em string com zero-padding
    step_str = str(step).zfill(5)

    # Salvar os perfis de pressão e velocidades
    press_filename = f"press_{step_str}.dat"
    velx_filename = f"velx_{step_str}.dat"
    vely_filename = f"vely_{step_str}.dat"

    write_file(press_filename, press, nx * ny)
    write_file(velx_filename, ux, nx * ny)
    write_file(vely_filename, uy, nx * ny)

    # Salvar os perfis de densidade para cada componente de fluido
    for s in range(nfluids):
        density_filename = f"density_comp{s}_{step_str}.dat"
        write_file(density_filename, rho[s], nx * ny)


def calculate_delta_pressure():
    """
    Calcula a pressão de Laplace e imprime densidades, diferenças de pressão, 
    raio da gota e tensão superficial.
    """
    rho_gas = 0.0
    rho_liq = 0.0
    press_gas = 0.0
    press_liq = 0.0

    # Pressão média do gás em um quadrado de 4 nós de malha
    for x in range(4):
        for y in range(4):
            rho_gas += rho[0,x,y]
            press_gas += press[x,y]

    # Pressão média do líquido em um quadrado de 4 nós de malha no centro
    for x in range(nx // 2 - 2, nx // 2 + 2):
        for y in range(ny // 2 - 2, ny // 2 + 2):
            rho_liq += rho[0,x,y]
            press_liq += press[x,y]

    # Calcula médias
    rho_gas /= 16.0
    rho_liq /= 16.0
    press_gas /= 16.0
    press_liq /= 16.0

    # Diferença de pressão e densidade média
    delta_press = press_liq - press_gas
    rho_av = 0.5 * (rho_gas + rho_liq)
    y = ny // 2
    rad = None

    # Determina o raio da gota
    for x in range(nx // 2):
        if rho[0,x,y] > rho_av:
            drho = rho[0,x,y] - rho[0,x-1,y]
            dx = (rho_av - rho[0,x-1,y]) / drho
            rad = (nx / 2.0 - x) + (1.0 - dx)
            break

    # Calcula a tensão superficial
    gamma = delta_press * rad

    # Imprime os resultados
    print(f"  Densities: rho_g = {rho_gas}, rho_l = {rho_liq}")
    print(f"  Pressure difference: dp = {delta_press}")
    print(f"  Droplet radius: r = {rad}")
    print(f"  Surface tension: gamma = {gamma}")

    return


# Função para imprimir status, fora do Numba
def print_status(step, elapsed_time):
    print("Running time step {}".format(step))
    print("Overall CPU time required: {:.2f} sec".format(elapsed_time))
    mlups = nx * ny * nsteps / (elapsed_time * 1e6)
    print("This corresponds to {:.2f} MLUPS".format(mlups))

# Função para criar animação, fora do Numba
def create_animation(output_dir):
    with imageio.get_writer("densidade_animacao.gif", mode="I", duration=0.1) as writer:
        frame_files = sorted(glob.glob("{}frame_*.png".format(output_dir)))  # Ordena os frames corretamente
        for filename in frame_files:
            image = imageio.imread(filename)
            writer.append_data(image)

    print("Animação salva como 'densidade_animacao.gif'")

def main():
    # Configurar cronômetro da simulação
    start = time.time()

    # Alocar memória para populações
    f1 = np.zeros((nfluids, nx, ny, npop))  # Populações antigas
    f2 = np.zeros((nfluids, nx, ny, npop))  # Populações novas
    # Inicializar domínio
    initialisation(f1, f2)

    # Loop principal
    for step in range(nsteps + 1):
        # Calcular campo de densidade
        compute_density(f1)

        # Calcular forças SC
        computeSCForces()

        # Calcular velocidade baricêntrica
        computeVelocity(f1)

        # Colidir e propagar
        push(f1, f2)

        # Trocar populações antigas e novas
        f1, f2 = f2, f1

        # Escrever dados em disco/console
        if step % noutput == 0:
            print_status(step, time.time() - start)
            computePressure()
            #write_profiles(step)
            #calculate_delta_pressure()
            rho_summed = np.einsum('sxy->xy', rho)
            rho_matriz = rho_summed.reshape(nx, ny)
            plt.figure(figsize=(6, 6))
            plt.imshow(rho_matriz, cmap="viridis", origin="lower")
            plt.colorbar(label="Densidade")
            plt.title("Frame {}".format(step))
#             plt.savefig("{}frame_{:03d}.png".format(output_dir, int(step / 10)))
            plt.savefig(os.path.join(output_dir, "frame_{:03d}.png".format(int(step / 10))))
            print(output_dir)
            plt.close()

    # Exibir estatísticas de desempenho
    finish = time.time()
    elapsed_time = finish - start
    print_status(nsteps, elapsed_time)  # Imprimir estatísticas no final

# Chamar a função principal
if __name__ == "__main__":
    main()

#Criar animação após a execução do cálculo
create_animation(output_dir)

### CODIGO MODIFICADO PARA INICIALIZAÇÃO ALEATÓRIA

Running time step 0
Overall CPU time required: 0.27 sec
This corresponds to 30.72 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 5
Overall CPU time required: 1.54 sec
This corresponds to 5.31 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 10
Overall CPU time required: 2.77 sec
This corresponds to 2.96 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 15
Overall CPU time required: 4.00 sec
This corresponds to 2.05 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 20
Overall CPU time required: 5.23 sec
This corresponds to 1.57 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 25
Overall CPU time required: 6.54 sec
This corresponds to 1.25 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 30
Overall CPU time required: 7.80 sec
This corresponds to 1.05 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 35
Overall CPU time required: 9.10 sec
This corresponds to 0.90 ML

Running time step 310
Overall CPU time required: 79.21 sec
This corresponds to 0.10 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 315
Overall CPU time required: 80.62 sec
This corresponds to 0.10 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 320
Overall CPU time required: 81.86 sec
This corresponds to 0.10 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 325
Overall CPU time required: 83.10 sec
This corresponds to 0.10 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 330
Overall CPU time required: 84.33 sec
This corresponds to 0.10 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 335
Overall CPU time required: 85.56 sec
This corresponds to 0.10 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 340
Overall CPU time required: 86.81 sec
This corresponds to 0.09 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 345
Overall CPU time required: 88.04 sec
This corre

Running time step 620
Overall CPU time required: 155.99 sec
This corresponds to 0.05 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 625
Overall CPU time required: 157.22 sec
This corresponds to 0.05 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 630
Overall CPU time required: 158.40 sec
This corresponds to 0.05 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 635
Overall CPU time required: 159.60 sec
This corresponds to 0.05 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 640
Overall CPU time required: 160.79 sec
This corresponds to 0.05 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 645
Overall CPU time required: 161.99 sec
This corresponds to 0.05 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 650
Overall CPU time required: 163.24 sec
This corresponds to 0.05 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 655
Overall CPU time required: 164.52 sec
Th

Running time step 930
Overall CPU time required: 231.50 sec
This corresponds to 0.04 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 935
Overall CPU time required: 232.70 sec
This corresponds to 0.04 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 940
Overall CPU time required: 233.90 sec
This corresponds to 0.04 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 945
Overall CPU time required: 235.10 sec
This corresponds to 0.03 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 950
Overall CPU time required: 236.29 sec
This corresponds to 0.03 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 955
Overall CPU time required: 237.50 sec
This corresponds to 0.03 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 960
Overall CPU time required: 238.69 sec
This corresponds to 0.03 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 965
Overall CPU time required: 239.89 sec
Th

Running time step 1235
Overall CPU time required: 305.53 sec
This corresponds to 0.03 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 1240
Overall CPU time required: 306.82 sec
This corresponds to 0.03 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 1245
Overall CPU time required: 308.04 sec
This corresponds to 0.03 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 1250
Overall CPU time required: 309.26 sec
This corresponds to 0.03 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 1255
Overall CPU time required: 310.46 sec
This corresponds to 0.03 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 1260
Overall CPU time required: 311.65 sec
This corresponds to 0.03 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 1265
Overall CPU time required: 312.89 sec
This corresponds to 0.03 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 1270
Overall CPU time required: 314.1

Running time step 1540
Overall CPU time required: 380.43 sec
This corresponds to 0.02 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 1545
Overall CPU time required: 381.63 sec
This corresponds to 0.02 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 1550
Overall CPU time required: 382.81 sec
This corresponds to 0.02 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 1555
Overall CPU time required: 384.01 sec
This corresponds to 0.02 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 1560
Overall CPU time required: 385.20 sec
This corresponds to 0.02 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 1565
Overall CPU time required: 386.40 sec
This corresponds to 0.02 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 1570
Overall CPU time required: 387.59 sec
This corresponds to 0.02 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 1575
Overall CPU time required: 388.7

Running time step 1845
Overall CPU time required: 455.40 sec
This corresponds to 0.02 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 1850
Overall CPU time required: 456.60 sec
This corresponds to 0.02 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 1855
Overall CPU time required: 457.80 sec
This corresponds to 0.02 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 1860
Overall CPU time required: 459.02 sec
This corresponds to 0.02 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 1865
Overall CPU time required: 460.26 sec
This corresponds to 0.02 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 1870
Overall CPU time required: 461.49 sec
This corresponds to 0.02 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 1875
Overall CPU time required: 462.69 sec
This corresponds to 0.02 MLUPS
/home/joao/Documents/PraticaLBM/Densidades
Running time step 1880
Overall CPU time required: 463.8

NameError: name 'imageio' is not defined

In [3]:
import numpy as np
import os
import time
import matplotlib.pyplot as plt


# Parâmetros do lattice
nx = 64  # número de nós ao longo do eixo x
ny = 64  # número de nós ao longo do eixo y
nsteps = 2000  # número de passos de tempo
noutput = 100  # intervalo de saída de dados (dados escritos em "/data")
nfluids = 2  # número de componentes do fluido (escolha 1 ou 2)
tauA = 1  # tempo de relaxação para o fluido A
tauB = 1  # tempo de relaxação para o fluido B (usado apenas se nfluids = 2)

# Parâmetros de Shan-Chen
gA = -4.7  # força de auto-interação do fluido A
gB = -4.7  # força de auto-interação do fluido B (usado apenas se nfluids = 2)
gAB = 6.  # força de interação dos fluidos A e B (usado apenas se nfluids = 2)
rho0 = 1.  # densidade de referência para o potencial pseudo (geralmente definido como 1)
rhol = 2.1  # densidade inicial do líquido
rhog = 0.15  # densidade inicial do gás
radius = 15.  # raio inicial da gota

# Parâmetros fixos; NÃO ALTERAR
npop = 9  # número de populações
cx =[0, 1, 0, -1, 0, 1, -1, -1, 1] # componentes x dos vetores do lattice
cy =[0, 0, 1,  0, -1, 1,  1, -1, -1]  # componentes y dos vetores do lattice
weight = [4./9., 1./9., 1./9., 1./9., 1./9., 1./36., 1./36., 1./36., 1./36.]


# Matrizes
g = np.zeros((nfluids, nfluids))  # forças de interação
rho = np.zeros((nfluids, nx, ny))  # densidade
press = np.zeros((nx, ny))  # pressão
ux = np.zeros((nx, ny))  # componente x da velocidade do fluido
uy = np.zeros((nx, ny))  # componente y da velocidade do fluido
psi_s = np.zeros((nx, ny))  # paralelizando calculos
psi_ss = np.zeros((nx, ny))  # paralelizando calculos
psinb = np.zeros((nx, ny))  # paralelizando calculos
# fxtemp=np.zeros((nx, ny))  # paralelizando calculos
# fytemp=np.zeros((nx, ny))  # paralelizando calculos
Fx = np.zeros((nfluids, nx, ny))  # componente x da força de Shan-Chen
Fy = np.zeros((nfluids, nx, ny))  # componente y da força de Shan-Chen
feq = np.zeros((nfluids, npop))  # populações de equilíbrio
forcing = np.zeros((nfluids, npop))  # populações de força
tau = np.zeros(nfluids)  # tempos de relaxação
f1 = np.zeros((nfluids, nx, ny, npop))  # populações (antigo)
f2 = np.zeros((nfluids, nx, ny, npop))  # populações (novo)

output_dir = os.path.join(os.path.expanduser("~"), "Documents", "PraticaLBM","Densidades")
os.makedirs(output_dir, exist_ok=True)



# Função para computar a densidade em todos os pontos

def compute_density(pop):
    global rho
    rho = np.zeros((nfluids, nx, ny))  # A matriz de densidade já é inicializada fora da função
    rho = np.einsum('sxyp->sxy', pop)

def computePressure():
    # Inicializa a matriz de pressão com zeros
    press.fill(0.0)
    
    # Calcula o termo de correção de forma vetorizada
    for s in range(nfluids):
        press += rho[s] / 3.0
        psi_s=rho0 * (1 - np.exp(-rho[s] / rho0))
        for ss in range(nfluids):
            psi_ss=rho0 * (1 - np.exp(-rho[ss] / rho0))
            press += (g[s, ss]*psi_s*psi_ss / 6.0)

def psi(densities):
    return rho0 * (1 - np.exp(-densities / rho0))
                    

def computeSCForces():
    # Inicializa as forças SC
    Fx.fill(0.0)
    Fy.fill(0.0)
    xvet=np.arange(nx)
    x2=np.zeros((nx,npop))
    yvet=np.arange(ny)
    y2=np.zeros((ny,npop))
    for x in range(nx):
        for i in range(npop):
            x2[x,i]=xvet[(x + cx[i]) % nx]
            y2[x,i]=yvet[(x + cy[i]) % ny]  ##como y=x nesse caso, usamos o mesmo loop
    # Computa as forças de Shan-Chen (SC)
    for x in range(nx):
        for y in range(ny):
            for s in range(nfluids):
                for ss in range(nfluids):
                    fxtemp=0
                    fytemp=0
                    for i in range(1, npop):  # Ignorar i=0 (sem força)
                        ##fx e fy temp tavam aqui
                        x2var=int(x2[x,i])
                        y2var=int(y2[y,i])
                        psinb[x,y] = rho0 * (1 - np.exp(-rho[s,x2var,y2var] / rho0))  # Psi do vizinho
                        fxtemp += weight[i] * cx[i] * psinb[x,y]
                        fytemp += weight[i] * cy[i] * psinb[x,y]                    
                    psiloc = rho0 * (1 - np.exp(-rho[s,x,y] / rho0)) # Psi local
                    Fx[s, x, y] -= g[s, ss] * psiloc * fxtemp
                    Fy[s, x, y] -= g[s, ss] * psiloc * fytemp

    
def computeTotalDensity(x, y):
    # Soma a densidade de todos os fluidos no ponto (x, y)
    return np.sum(rho[:, x, y])


def computePressure():
    # Inicializa a pressão
    psi = rho0 * (1 - np.exp(-rho / rho0))  # Calcula psi para todos os fluidos de uma vez
    press = np.einsum('sxy->xy',rho)/3.0  # Soma as densidades divididas por 3.0

    # Soma contribuições para a pressão com operações vetorizadas
    press[:, :] += np.sum(g[:, :, None, None] * psi[:, None, :, :] * psi[None, :, :, :], axis=(0, 1)) / 6.0
                    


def computeVelocity(pop):
    ux.fill(0.0)
    uy.fill(0.0)
    for s in range(nfluids):
        # Calcula contribuições da população
        ux[:,:] += (pop[s, :, :, 1] - pop[s, :, :, 3] +
                     pop[s, :, :, 5] - pop[s, :, :, 6] -
                     pop[s, :, :, 7] + pop[s, :, :, 8])

        uy[:, :] += (pop[s, :, :, 2] - pop[s, :, :, 4] +
                     pop[s, :, :, 5] + pop[s, :, :, 6] -
                     pop[s, :, :, 7] - pop[s, :, :, 8])

        # Adiciona força
        ux[:, :] += 0.5 * Fx[s, :, :]
        uy[:, :] += 0.5 * Fy[s, :, :]

# Normaliza pela densidade total
    for x in range(nx):
        for y in range(ny):
            dens = computeTotalDensity(x,y)
            ux[x, y] /= dens
            uy[x, y] /= dens

            

def equilibrium(s,x,y):
    dens = rho[s,x,y]
    vx = ux[x,y]
    vy = uy[x,y]
    usq = vx * vx + vy * vy
  
    feq[s][0] = weight[0] * dens * (1. - 1.5 * usq)
    feq[s][1] = weight[1] * dens * (1. + 3. * vx + 4.5 * vx * vx - 1.5 * usq)
    feq[s][2] = weight[2] * dens * (1. + 3. * vy + 4.5 * vy * vy - 1.5 * usq)
    feq[s][3] = weight[3] * dens * (1. - 3. * vx + 4.5 * vx * vx - 1.5 * usq)
    feq[s][4] = weight[4] * dens * (1. - 3. * vy + 4.5 * vy * vy - 1.5 * usq)
    feq[s][5] = weight[5] * dens * (1. + 3. * ( vx + vy) + 4.5 * ( vx + vy) * ( vx + vy) - 1.5 * usq)
    feq[s][6] = weight[6] * dens * (1. + 3. * (-vx + vy) + 4.5 * (-vx + vy) * (-vx + vy) - 1.5 * usq)
    feq[s][7] = weight[7] * dens * (1. + 3. * (-vx - vy) + 4.5 * ( vx + vy) * ( vx + vy) - 1.5 * usq)
    feq[s][8] = weight[8] * dens * (1. + 3. * ( vx - vy) + 4.5 * ( vx - vy) * ( vx - vy) - 1.5 * usq)
  

  
def initialisation(f1, f2):
    # Definir viscosidade
    tau[0] = tauA
    if nfluids == 2:
        tau[1] = tauB

    # Definir força de interação
    g[0, 0] = gA
    if nfluids == 2:
        g[1, 1] = gB
        g[0, 1] = gAB
        g[1, 0] = gAB

    # Inicialização para 1 fluido
    if nfluids == 1:
        x_grid, y_grid = np.meshgrid(np.arange(nx), np.arange(ny), indexing="ij")
        distances = (y_grid - ny / 2.0) ** 2 + (x_grid - nx / 2.0) ** 2
        rho[0] = np.where(distances <= radius ** 2, rhol, rhog)

    # Inicialização para 2 fluidos
    elif nfluids == 2:
        mid_y = ny // 2
        mid_x = nx // 2  # Determinando o ponto central na direção x
        
        # Inicializando os fluidos A e B em cada lado da bolha
        for y in range(ny):
            for x in range(nx):
                # Distância de cada ponto ao centro
                dist_to_center = np.sqrt((x - mid_x) ** 2 + (y - mid_y) ** 2)

                # Se dentro do raio da bolha, alocar os fluidos A e B
                if dist_to_center <= radius:
                    if x < mid_x:  # Lado esquerdo
                        rho[0, x, y] = rhol  # Fluido A (líquido)
                        rho[1, x, y] = 0     # Fluido B (não tem)
                    else:  # Lado direito
                        rho[0, x, y] = 0     # Fluido A (não tem)
                        rho[1, x, y] = rhol  # Fluido B (líquido)
                else:
                    rho[0, x, y] = rhog  # Fora da bolha, atribui gás
                    rho[1, x, y] = 0     # Fora da bolha


    # Configurar o campo de velocidade inicial (velocidade zero em todos os lugares)
    ux.fill(0.0)
    uy.fill(0.0)

    # Calcular as distribuições de equilíbrio e inicializar as populações
    for s in range(nfluids):
        for y in range(ny):
            for x in range(nx):
                equilibrium(s, x, y)
                f1[s, x, y, :] = feq[s, :]
                f2[s, x, y, :] = feq[s, :]    
    
def push(f1, f2):
    for s in range(nfluids):
        omega = 1.0 / tau[s]

        for x in range(nx):  
            for y in range(ny):  
                for i in range(npop):
                    forcing[s,i] = weight[i] * (1.0 - 0.5 * omega) * ((3.0 * (cx[i] - ux[x, y]) + 9.0 * cx[i] * (cx[i] * ux[x, y] + cy[i] * uy[x, y])) * Fx[s, x, y] +
                    (3.0 * (cy[i] - uy[x, y]) + 9.0 * cy[i] * (cx[i] * ux[x, y] + cy[i] * uy[x, y])) * Fy[s, x, y])

                # Compute equilibrium distributions
                equilibrium(s, x, y)

                # Collide and propagate
                for i in range(npop):
                    x2 = (x + cx[i] + nx) % nx  # Periodic boundary in x
                    y2 = (y + cy[i] + ny) % ny  # Periodic boundary in y
                    f2[s, x2, y2, i] = f1[s, x, y, i] * (1.0 - omega) + feq[s, i] * omega + forcing[s,i]

    return

                 
def write_file(name, data, n):
    #Função auxiliar para salvar dados em disco.
    name = os.path.expanduser(os.path.join("~", "Documents", "PraticaLBM", "2 Fluidos", name))
    # Verifique se o diretório existe, caso contrário, crie-o
    os.makedirs(os.path.dirname(name), exist_ok=True)
    with open(name, "w") as fout:
        for y in range(ny):
            for x in range(nx):
                fout.write(f"{data[y * nx + x]} ")
            fout.write("\n")
        fout.write("\n")


def write_profiles(step):
    """
    Função para salvar os perfis (densidade, pressão, velocidades) no disco.
    """
    # Converter o número do passo em string com zero-padding
    step_str = str(step).zfill(5)

    # Salvar os perfis de pressão e velocidades
    press_filename = f"press_{step_str}.dat"
    velx_filename = f"velx_{step_str}.dat"
    vely_filename = f"vely_{step_str}.dat"

    write_file(press_filename, press, nx * ny)
    write_file(velx_filename, ux, nx * ny)
    write_file(vely_filename, uy, nx * ny)

    # Salvar os perfis de densidade para cada componente de fluido
    for s in range(nfluids):
        density_filename = f"density_comp{s}_{step_str}.dat"
        write_file(density_filename, rho[s], nx * ny)


def calculate_delta_pressure():
    """
    Calcula a pressão de Laplace e imprime densidades, diferenças de pressão, 
    raio da gota e tensão superficial.
    """
    rho_gas = 0.0
    rho_liq = 0.0
    press_gas = 0.0
    press_liq = 0.0

    # Pressão média do gás em um quadrado de 4 nós de malha
    for x in range(4):
        for y in range(4):
            rho_gas += rho[0,x,y]
            press_gas += press[x,y]

    # Pressão média do líquido em um quadrado de 4 nós de malha no centro
    for x in range(nx // 2 - 2, nx // 2 + 2):
        for y in range(ny // 2 - 2, ny // 2 + 2):
            rho_liq += rho[0,x,y]
            press_liq += press[x,y]

    # Calcula médias
    rho_gas /= 16.0
    rho_liq /= 16.0
    press_gas /= 16.0
    press_liq /= 16.0

    # Diferença de pressão e densidade média
    delta_press = press_liq - press_gas
    rho_av = 0.5 * (rho_gas + rho_liq)
    y = ny // 2
    rad = None

    # Determina o raio da gota
    for x in range(nx // 2):
        if rho[0,x,y] > rho_av:
            drho = rho[0,x,y] - rho[0,x-1,y]
            dx = (rho_av - rho[0,x-1,y]) / drho
            rad = (nx / 2.0 - x) + (1.0 - dx)
            break

    # Calcula a tensão superficial
    gamma = delta_press * rad

    # Imprime os resultados
    print(f"  Densities: rho_g = {rho_gas}, rho_l = {rho_liq}")
    print(f"  Pressure difference: dp = {delta_press}")
    print(f"  Droplet radius: r = {rad}")
    print(f"  Surface tension: gamma = {gamma}")

    return


# Função para imprimir status, fora do Numba
def print_status(step, elapsed_time):
    print("Running time step {}".format(step))
    print("Overall CPU time required: {:.2f} sec".format(elapsed_time))
    mlups = nx * ny * nsteps / (elapsed_time * 1e6)
    print("This corresponds to {:.2f} MLUPS".format(mlups))

# Função para criar animação, fora do Numba
def create_animation(output_dir):
    with imageio.get_writer("densidade_animacao.gif", mode="I", duration=0.1) as writer:
        frame_files = sorted(glob.glob("{}frame_*.png".format(output_dir)))  # Ordena os frames corretamente
        for filename in frame_files:
            image = imageio.imread(filename)
            writer.append_data(image)

    print("Animação salva como 'densidade_animacao.gif'")

def main():
    # Configurar cronômetro da simulação
    start = time.time()

    # Alocar memória para populações
    f1 = np.zeros((nfluids, nx, ny, npop))  # Populações antigas
    f2 = np.zeros((nfluids, nx, ny, npop))  # Populações novas
    # Inicializar domínio
    initialisation(f1, f2)

    # Loop principal
    for step in range(nsteps + 1):
        # Calcular campo de densidade
        compute_density(f1)

        # Calcular forças SC
        computeSCForces()

        # Calcular velocidade baricêntrica
        computeVelocity(f1)

        # Colidir e propagar
        push(f1, f2)

        # Trocar populações antigas e novas
        f1, f2 = f2, f1

        # Escrever dados em disco/console
        if step % noutput == 0:
            print_status(step, time.time() - start)
            computePressure()
            #write_profiles(step)
            calculate_delta_pressure()
            rho_summed = np.einsum('sxy->xy', rho)
            rho_matriz = rho_summed.reshape(nx, ny)
            plt.figure(figsize=(6, 6))
            plt.imshow(rho_matriz, cmap="viridis", origin="lower")
            plt.colorbar(label="Densidade")
            plt.title("Frame {}".format(step))
            plt.savefig("{}frame_{:03d}.png".format(output_dir, int(step / 10)))
            plt.close()

    # Exibir estatísticas de desempenho
    finish = time.time()
    elapsed_time = finish - start
    print_status(nsteps, elapsed_time)  # Imprimir estatísticas no final

# Chamar a função principal
if __name__ == "__main__":
    main()


Running time step 0
Overall CPU time required: 0.68 sec
This corresponds to 12.06 MLUPS
  Densities: rho_g = 0.14999999999999997, rho_l = 1.05
  Pressure difference: dp = 0.0
  Droplet radius: r = 15.76923076923077
  Surface tension: gamma = 0.0


/tmp/ipykernel_4273/781145626.py:102: RuntimeWarning: overflow encountered in exp
  psinb[x,y] = rho0 * (1 - np.exp(-rho[s,x2var,y2var] / rho0))  # Psi do vizinho
/tmp/ipykernel_4273/781145626.py:106: RuntimeWarning: invalid value encountered in scalar subtract
  Fx[s, x, y] -= g[s, ss] * psiloc * fxtemp
/tmp/ipykernel_4273/781145626.py:107: RuntimeWarning: invalid value encountered in scalar subtract
  Fy[s, x, y] -= g[s, ss] * psiloc * fytemp
/tmp/ipykernel_4273/781145626.py:104: RuntimeWarning: invalid value encountered in scalar multiply
  fytemp += weight[i] * cy[i] * psinb[x,y]
/tmp/ipykernel_4273/781145626.py:103: RuntimeWarning: invalid value encountered in scalar multiply
  fxtemp += weight[i] * cx[i] * psinb[x,y]
/tmp/ipykernel_4273/781145626.py:105: RuntimeWarning: overflow encountered in exp
  psiloc = rho0 * (1 - np.exp(-rho[s,x,y] / rho0)) # Psi local
/tmp/ipykernel_4273/781145626.py:104: RuntimeWarning: invalid value encountered in scalar add
  fytemp += weight[i] * cy[i

Running time step 100
Overall CPU time required: 59.75 sec
This corresponds to 0.14 MLUPS


TypeError: unsupported operand type(s) for *: 'float' and 'NoneType'